# Interactive Sentiment Analyser For Movie Reviews Using NLTK-Python-TFlearn (Tensorflow)

In [28]:
# Create My own Lexicon for data preprocessing 

import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
from tqdm import tqdm

word_list = []

lem = WordNetLemmatizer()

with open('D:\\sentiment_analysis\\pos.txt') as f:
    content = f.readlines()

for i in tqdm(range(0,len(content))):
    tokens = (word_tokenize(content[i]))
    for word in tokens:
        word = lem.lemmatize(word)
        if word not in word_list:
            word_list.append(word)

                  
with open('D:\\sentiment_analysis\\neg.txt') as f:
    content = f.readlines()

for i in tqdm(range(0,len(content))):
    tokens = word_tokenize(content[i])   
    for word in tokens:
        word = lem.lemmatize(word)
        if word not in word_list:
            word_list.append(word)

        
array = np.array(word_list)
np.save('dict.npy',array)

100%|█████████████████████████████████████████████████████████████████████████████| 5331/5331 [00:05<00:00, 922.49it/s]


In [33]:

# Preprocess the data set as per the lexicon above.

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
from tqdm import tqdm
from numba import jit


lexicon = np.load('dict.npy')

# for positive sentiment data-set.

with open('D:\\sentiment_analysis\\pos.txt') as f:
    content = f.readlines()

pos = np.zeros((len(content),int(lexicon.size)), dtype=np.int)    

for i in tqdm(range(0,len(content))):
    sent = content[i]
    sent = word_tokenize(sent)
    for word in sent:
        word = lem.lemmatize(word)
        for j in range(0,int(lexicon.size)):
            if str(word) == str(lexicon[j]):
                pos[i][j] +=1


np.save('pos-senti-preprocessed.npy',pos)




100%|██████████████████████████████████████████████████████████████████████████████| 5331/5331 [33:05<00:00,  3.12it/s]


'\nfunction()\n'

In [35]:

# for negative sentiment data-set.

lexicon = np.load('dict.npy')


with open('D:\\sentiment_analysis\\neg.txt') as f:
    content = f.readlines()

neg = np.zeros((len(content),int(lexicon.size)), dtype=np.int)    

for i in tqdm(range(0,len(content))):
    sent = content[i]
    sent = word_tokenize(sent)
    for word in sent:
        word = lem.lemmatize(word)
        for j in range(0,int(lexicon.size)):
            if str(word) == str(lexicon[j]):
                neg[i][j] +=1


np.save('neg-senti-preprocessed.npy',neg)

100%|██████████████████████████████████████████████████████████████████████████████| 5331/5331 [32:22<00:00,  3.15it/s]


In [49]:
# combine the data after preprocessing
import numpy as np
from tqdm import tqdm

final = []

pos = np.load('pos-senti-preprocessed.npy')
pos_one_hot = np.array([1,0])

for i in tqdm(range(0,5331)):
    final = np.append(final,(pos[i],pos_one_hot))
    
neg = np.load('neg-senti-preprocessed.npy')
neg_one_hot = np.array([0,1])

for i in tqdm(range(0,5331)):
    final = np.append(final,(neg[i],neg_one_hot))

print(final)
final = final.reshape(10662,2)
np.random.shuffle(final)
np.save('Final-preprocessed-sentiment-data',final)

100%|███████████████████████████████████████████████████████████████████████████| 5331/5331 [00:00<00:00, 15257.82it/s]


[array([2, 1, 1, ..., 0, 0, 0]) array([1, 0])
 array([3, 0, 1, ..., 0, 0, 0]) ..., array([0, 1])
 array([0, 0, 1, ..., 0, 0, 0]) array([0, 1])]


# Training the Neural Network

In [2]:
import tensorflow as tf
import tflearn 
import numpy as np
from tqdm import tqdm
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import os

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [4]:
tf.reset_default_graph()

IMG_SIZE = 1
LR = 1e-3

MODEL_NAME = 'NLP-sentiment-analysis-CNN'

array = np.load('Final-preprocessed-sentiment-data-shuffled.npy')
array = array.reshape(10662,2)

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE,18669], name='input')
'''
convnet =  tflearn.lstm(convnet, 1024, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 512, activation='relu')
'''
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 128, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


X = np.array([i[0] for i in array]).reshape(-1,IMG_SIZE,IMG_SIZE,18669)
Y = [i[1] for i in array]

model.fit({'input': X}, {'targets': Y}, n_epoch=10, snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

model.save(MODEL_NAME)




Training Step: 1669  | total loss: 0.08413 | time: 7.491s
| Adam | epoch: 010 | loss: 0.08413 - acc: 0.9940 -- iter: 10624/10662
Training Step: 1670  | total loss: 0.07613 | time: 7.534s
| Adam | epoch: 010 | loss: 0.07613 - acc: 0.9946 -- iter: 10662/10662
--
INFO:tensorflow:C:\Users\Shekhar Shiroor\NLP-sentiment-analysis-CNN is not in all_model_checkpoint_paths. Manually adding it.


# Test for the custom data from the User

In [1]:
import tensorflow as tf
import tflearn 
import numpy as np
from tqdm import tqdm
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [2]:

tf.reset_default_graph()

IMG_SIZE = 3
LR = 1e-3

MODEL_NAME = 'NLP-sentiment-analysis'

convnet = input_data(shape=[None, IMG_SIZE, 6223], name='input')

convnet =  tflearn.lstm(convnet, 1024, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 512, activation='relu')

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 128, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


lexicon = np.load('dict.npy')

lem = WordNetLemmatizer()


while True:
    print('\nEnter the sentence for testing its sentiment by Neural Neywork Model !...\n')
    string = str(input())
    arr = np.zeros((1,int(lexicon.size)), dtype=np.int)
    sent = word_tokenize(string)
    for word in sent:
        word = lem.lemmatize(word)
        for j in range(0,int(lexicon.size)):
            if str(word) == str(lexicon[j]):
                arr[0][j] +=1
    
    arr = arr.reshape(-1,3,6223)
    model_out = model.predict(arr)
    
    if np.argmax(model_out) == 0:
        print('\n The Sentiment is Positive ')
    else:
        print('\n The Sentiment is Negative ')
        
    print('\n Would like to test on another sentence ? \n')
    t = str(input())
    if t == 'N' or t == 'No' or t == 'no' or t == 'n':
        break

model loaded!

Enter the sentence for testing its sentiment by Neural Neywork Model !...

I am happy with the movie

 The Sentiment is Positive 

 Would like to test on another sentence ? 

y

Enter the sentence for testing its sentiment by Neural Neywork Model !...

I am not that happy with the movie

 The Sentiment is Positive 

 Would like to test on another sentence ? 

y

Enter the sentence for testing its sentiment by Neural Neywork Model !...

The movie was good but could be better

 The Sentiment is Negative 

 Would like to test on another sentence ? 

n


In [3]:
# NLP By Using CNN Model

tf.reset_default_graph()

IMG_SIZE = 1
LR = 1e-3

MODEL_NAME = 'NLP-sentiment-analysis-CNN'

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE,18669], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 128, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


lexicon = np.load('dict.npy')

lem = WordNetLemmatizer()


while True:
    print('\nEnter the sentence for testing its sentiment by Neural Neywork Model !...\n')
    string = str(input())
    arr = np.zeros((1,int(lexicon.size)), dtype=np.int)
    sent = word_tokenize(string)
    for word in sent:
        word = lem.lemmatize(word)
        for j in range(0,int(lexicon.size)):
            if str(word) == str(lexicon[j]):
                arr[0][j] +=1
    
    arr = arr.reshape(-1,IMG_SIZE,IMG_SIZE,18669)
    model_out = model.predict(arr)
    
    if np.argmax(model_out) == 0:
        print('\n The Sentiment is Positive ')
    else:
        print('\n The Sentiment is Negative ')
        
    print('\n Would like to test on another sentence ? \n')
    t = str(input())
    if t == 'N' or t == 'No' or t == 'no' or t == 'n':
        break

model loaded!

Enter the sentence for testing its sentiment by Neural Neywork Model !...

I am not that happy with the movie

 The Sentiment is Positive 

 Would like to test on another sentence ? 

yes

Enter the sentence for testing its sentiment by Neural Neywork Model !...

The movie could have been better

 The Sentiment is Negative 

 Would like to test on another sentence ? 

yes

Enter the sentence for testing its sentiment by Neural Neywork Model !...

The movie was up to the mark

 The Sentiment is Negative 

 Would like to test on another sentence ? 

n


# Visualization of data in Tensorboard (PCA and t-SNE)

In [4]:
array = np.load('Final-preprocessed-sentiment-data-shuffled.npy')

file = open( 'metadata-NLP.tsv', 'w')
file.write('Name\tClass\n')

for i in tqdm(range(0,2000)):
    temp = array[i][1]
    if(temp[0]==1):
        file.write('%s\n' % ('Positive'))
    else:
        file.write('%s\n' % ('Negative'))
        
file.close()        

100%|██████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 170517.49it/s]


In [3]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
from tqdm import tqdm


array = np.load('Final-preprocessed-sentiment-data-shuffled.npy')
temp = []

for i in tqdm(range(0,2000)):
    temp = np.append(temp,array[i][0])

temp = temp.reshape(2000,18669)    

embedding_variable = tf.Variable(temp, name='embedding')

summary_writer = tf.summary.FileWriter('./log-NLP')
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_variable.name
embedding.metadata_path = 'metadata-NLP.tsv'
projector.visualize_embeddings(summary_writer, config)


session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

saver = tf.train.Saver()
saver.save(session, './log-NLP/model.ckpt', 0)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [03:36<00:00,  5.27it/s]


'./log-NLP/model.ckpt-0'